## Visualize groundplane alignement

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
import os, sys
import shutil

import cv2
import numpy as np

BASEPATH = os.getcwd().split('code')[-2]
CODEPATH = os.path.join(BASEPATH, 'code')
DATAPATH = os.path.join(BASEPATH, 'data')

sys.path.append(CODEPATH)

from matplotlib import pyplot as plt

from configs.arguments import get_config_dict
from utils.coordinate_utils import project_to_ground_plane_cv2, triangulate_point, project_points
from utils.multiview_utils import Camera, Calibration
from utils.metadata_utils import get_cam_names
from utils.multiview_utils import MultiviewVids

%matplotlib inline

sys.argv = [""]
if "pomelo_cfg" in os.environ:
    sys.argv.append('-cfg')
    config_path = os.path.abspath(os.environ["pomelo_cfg"])
    sys.argv.append(config_path)

try:
    config = get_config_dict()
except:
    log.warning("No config file found. Using default values.")
    config = {}

In [ ]:
mvvids = MultiviewVids(newest=False)

max_frame = np.min([10, mvvids.get_max_frame_id() - 1])
step = 2
base_frames = {}

frame_ids = list(np.arange(0, max_frame, step))

base_frames = mvvids.extract_mv(frame_ids, undistort = True)


In [ ]:
# Visualize groundplane

for cam in mvvids.cams:
    fig = plt.figure(figsize=(10,10))
    img = base_frames[cam.name][0]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    plt.imshow(img)
    plt.show()

Adjust output_img_size especially aspect ratio such that the area of interest take the most space.
Once satisfied in the results save the value in train_config.yaml hm size.
The range of value should be in the 100-400 range depending on gpu memory size.

In [ ]:
rect, _ = mvvids.get_bounding_box()

output_img_size = (1000, 1500)
padding_percent = 0

ground_imgs = []

for cam in mvvids.cams:
    fig = plt.figure(figsize=(10,10))
    img = base_frames[cam.name][0]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    input_img_size = base_frames[cam.name][0].shape[:2]

    H = cam.get_ground_plane_homography(input_img_size=input_img_size, output_img_size = output_img_size, 
                                        bounding_box = rect, padding_percent = padding_percent)

    new_img = project_to_ground_plane_cv2(img, H, output_img_size)
    ground_imgs.append(new_img)
    plt.imshow(new_img)
    plt.show()

#Overal images to see alignement
plt.figure(figsize=(10,10))
for i, img in enumerate(ground_imgs):
    plt.imshow(img, alpha=1-i/len(ground_imgs))

plt.show()

## Check groundplane alignement

In [ ]:
from matplotlib.patches import Polygon
import ipywidgets as widgets
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
point_3ds = []

Select a matching ground point between at least two views. 

In [ ]:
plt.close('all')

ann_points = {cam.name: [] for cam in mvvids.cams}

# create dropdown widget with camera names as options
cam_dropdown = widgets.Dropdown(
    options=[cam.name for cam in mvvids.cams],
    value=mvvids.cams[0].name,
    description='Camera:',
    disabled=False,
)

def onclick(event, cam):
    ax.plot(event.xdata, event.ydata, 'rx', markersize=10)
    ax.annotate(f'{len(ann_points[cam.name])}', xy=(event.xdata+10, event.ydata+7), textcoords='data')
    ann_points[cam.name].append(([event.xdata, event.ydata]))
    
    fig.canvas.draw()
    pass

def update_cam(change):
    ax.cla()
    global cam
    cam_name = change.new
    cam = next(cam for cam in mvvids.cams if cam.name == cam_name)
    ax.set_title(f"Camera: {cam.name}")
    ax.imshow(base_frames[cam.name][0])
    for i, point in enumerate(ann_points[cam.name]):
        ax.plot(point[0], point[1], 'rx', markersize=10)
        ax.annotate(f'{i}', xy=(point[0]+10, point[1]+7), textcoords='data')
    fig.canvas.draw()
    pass

cam_dropdown.observe(update_cam, names='value')

cam = mvvids.cams[0]
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(base_frames[cam.name][0])

for i, point in enumerate(ann_points[cam.name]):
    ax.plot(point[0], point[1], 'rx', markersize=10)
    ax.annotate(f'{i}', xy=(point[0]+10, point[1]+7), textcoords='data')
    
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

cid = fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, cam))

display(cam_dropdown)
# display(fig)

In [ ]:
plt.close('all')


for cam_name, points in ann_points.items():
    print(cam_name,":")
    print("\t",points)
    if len(points) > 1:
        print("Only select one point at a time, please re-execute previous cell before continuing")

If the scene is aligned properly we expect point on the floor to have there third coordinate (z) to be zero.
If it's not the case we'll need to realign the reconstruction.

In [ ]:
obs_2d = []
calibs = []

for cam_name, points in ann_points.items():
    if len(points) == 0:
        continue

    obs_2d.append(points[0])
    calibs.append(mvvids.get_cam_by_name(cam_name).get_calib())


triangulated_point = triangulate_point(obs_2d, calibs)
print(triangulated_point)
                  
point_3ds.append(triangulated_point.squeeze())


### You can repeat the last three cell to add more ground matching point. The more the better.

In [ ]:
# Visualize groundplane
plt.close('all')

%matplotlib inline

all_reproj_2d = []
for cam in mvvids.cams:
    fig = plt.figure(figsize=(10,10))
    img = base_frames[cam.name][0]
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.imshow(img)

    reproj_2d, _ = cam.convert_to_camera_frame(np.vstack(point_3ds))
    all_reproj_2d.append(reproj_2d)
    plt.scatter(reproj_2d[:,0],reproj_2d[:,1], c="red")
    plt.show()

### Project the point on the ground using the groundplane assumption. If the ground is correctly alligned the point should overlap or be very close to each other, if it's not the case proceed with realignement below.

In [ ]:

all_point_g = []

for i, cam in enumerate(mvvids.cams):
    fig = plt.figure(figsize=(10,10))
    img_ground = ground_imgs[i]

    input_img_size = base_frames[cam.name][0].shape[:2]

    H = cam.get_ground_plane_homography(input_img_size=input_img_size, output_img_size = output_img_size, 
                                        bounding_box = rect, padding_percent = padding_percent)

    point_cam_g = project_points(all_reproj_2d[i], np.linalg.inv(H))
    print(point_cam_g,point_cam_g.shape)
    all_point_g.append(point_cam_g.squeeze())
    
    plt.imshow(img_ground)
    plt.scatter(point_cam_g[:,0], point_cam_g[:,1], c="red")
    
    plt.show()

#Overal images to see alignement
plt.figure(figsize=(10,10))
for i, img in enumerate(ground_imgs):
    plt.imshow(img, alpha=1-i/len(ground_imgs))


all_point_g = np.vstack(all_point_g)


plt.scatter(all_point_g[:,0], all_point_g[:,1])
plt.show()

In [ ]:
z_offsets = np.vstack(point_3ds)[:,2].mean()

print(f"z_offsets = {z_offsets} if the value is close to zero you can stop here the calibration is complete, otherwise proceed")

## Scaling reconstruction to match real world

Select two points on the ground which the distance between the two (in cm) is roughly known

In [ ]:
%matplotlib widget

plt.close('all')

world_scale_point = {cam.name: [] for cam in mvvids.cams}

# create dropdown widget with camera names as options
cam_dropdown = widgets.Dropdown(
    options=[cam.name for cam in mvvids.cams],
    value=mvvids.cams[0].name,
    description='Camera:',
    disabled=False,
)

def onclick(event, cam):
    ax.plot(event.xdata, event.ydata, 'rx', markersize=10)
    ax.annotate(f'{len(world_scale_point[cam.name])}', xy=(event.xdata+10, event.ydata+7), textcoords='data')
    world_scale_point[cam.name].append(([event.xdata, event.ydata]))
    
    fig.canvas.draw()
    pass

def update_cam(change):
    ax.cla()
    global cam
    cam_name = change.new
    cam = next(cam for cam in mvvids.cams if cam.name == cam_name)
    ax.set_title(f"Camera: {cam.name}")
    ax.imshow(base_frames[cam.name][0])
    for i, point in enumerate(world_scale_point[cam.name]):
        ax.plot(point[0], point[1], 'rx', markersize=10)
        ax.annotate(f'{i}', xy=(point[0]+10, point[1]+7), textcoords='data')
    fig.canvas.draw()
    pass

cam_dropdown.observe(update_cam, names='value')

cam = mvvids.cams[0]
fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(base_frames[cam.name][0])

for i, point in enumerate(world_scale_point[cam.name]):
    ax.plot(point[0], point[1], 'rx', markersize=10)
    ax.annotate(f'{i}', xy=(point[0]+10, point[1]+7), textcoords='data')
    
fig.subplots_adjust(left=0, right=1, bottom=0, top=1)

cid = fig.canvas.mpl_connect('button_press_event', lambda event: onclick(event, cam))

display(cam_dropdown)
# display(fig)

In [ ]:
plt.close('all')
%matplotlib inline

world_scale_point = {k:v for k,v in world_scale_point.items() if len(v) > 0}

distance_between_point = 200

In [ ]:
cam = mvvids.get_cam_by_name(list(world_scale_point.keys())[0])
points = np.vstack(list(world_scale_point.values())[0])

H = cam.get_ground_plane_homography(input_img_size=input_img_size, output_img_size = output_img_size, 
                                    bounding_box = rect, padding_percent = padding_percent)

point_scale_ground = project_points(points, np.linalg.inv(H))

dist_measured = np.linalg.norm(point_scale_ground[0]-point_scale_ground[1])
print(f"distance measure {dist_measured} should be {distance_between_point}")

In [ ]:
scale_rec = distance_between_point / dist_measured
print(f"Estimated reconstruction scaling factor {scale_rec}")

# Updating calibration with z-offset and scaling information

In [ ]:
from utils import pairs_from_sequence, pairs_from_360

tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np
import pycolmap
import copy

from matplotlib import pyplot as plt

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_exhaustive
)

from hloc.localize_sfm import QueryLocalizer, pose_from_cluster
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d
import pycolmap




from configs.arguments import get_config_dict
from utils.colmap_utils import add_camera_and_image, camera_calibs_from_colmap
from utils.colmap_visualization import plot_reconstruction
from utils.multiview_utils import Camera, Calibration
from utils.metadata_utils import get_cam_names


%matplotlib inline

In [ ]:
data_root = Path(config.get('main', {}).get('data_root', DATAPATH))

    
RESTART = config["calibration"]["force_reconstruction"]

images = data_root / "0-calibration/images"
outputs = data_root / "0-calibration/outputs/"

sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"
model_dir = outputs / 'full_reconstruction/'

rec_already_exists = (outputs / "full_reconstruction").exists()

In [ ]:
model = pycolmap.Reconstruction(outputs / "full_reconstruction")
references = [p.relative_to(images).as_posix() for p in images.rglob('*.jpg') if '360' in p.parent.name]
query = [p.relative_to(images).as_posix() for p in images.rglob('*.jpg') if '360' not in p.parent.name]

In [ ]:
# camera = pycolmap.infer_camera_from_image(images / query)
ref_ids = [model.find_image_with_name(r).image_id for r in references if model.find_image_with_name(r) is not None]
conf = {
    "estimation": {"ransac": {"max_error": 12}},
    "refinement": {"refine_focal_length": False, "refine_extra_params": False},
}

localizer = QueryLocalizer(model, conf)
rets = []
logs = []
for query_ in query:
    camera = pycolmap.infer_camera_from_image(images / query_)
    camera.params = [640, 640, 340, 0]
    ret, pose_log = pose_from_cluster(localizer, query_, camera, ref_ids, features, matches)
    ret['image'] = query_
    # add results to reconstruction
    try:
        add_camera_and_image(model, ret['camera'], ret["cam_from_world"], ret["image"])
    except ValueError as e:
        log.error(e)

    rets.append(ret)
    logs.append(pose_log)

    print(f'found {ret["num_inliers"]}/{len(ret["inliers"])} inlier correspondences.')


temp_model = copy.deepcopy(model)

# Transform the model
temp_model.transform(pycolmap.Sim3d(1, pycolmap.Rotation3d([-0.7071068, 0, 0, 0.7071068]), [0, 0, 0]))

# Filter out outlier and use bbox to set initial z=0
bbox = temp_model.compute_bounding_box(0.01, 0.99)
temp_model.transform(pycolmap.Sim3d(1, pycolmap.Rotation3d([0, 0, 0, 1]), [0, 0, -temp_model.compute_bounding_box(0.01, 0.99)[0][2]]))
temp_model.transform(pycolmap.Sim3d(100, pycolmap.Rotation3d([0, 0, 0, 1]), [0, 0, 0]))

## Updating and saving extrinsic calibration

In [ ]:
if not 'z_offsets' in locals():
  z_offsets = 0

if not 'scale_rec' in locals():
  scale_rec = 1

print(f"Updating reconstruction with scaling of {scale_rec} and z offset of {z_offsets}")
temp_model.transform(pycolmap.Sim3d(scale_rec, pycolmap.Rotation3d([0, 0, 0, 1]), [0, 0, -z_offsets]))

In [ ]:
cameras = camera_calibs_from_colmap(images, temp_model, save=True)

print("Extrinsic camera calibration completed. Restart the notebook to verify again groundplane alignement")

# Visualize the reconstruction
plot_reconstruction(temp_model, save_path=outputs / "360-reconstruction_w_static_updated.html")

print(f"Scene visualization completed. Results saved in {outputs / '360-reconstruction_w_static_updated.html'}")